In [2]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision=4)

In [5]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])

#Iterate over a dataset making a python iteration using iter, next consumes the element
it = iter(dataset)
print(next(it).numpy())
print(next(it).numpy())

#Perform an action on all elements in the dataset
print(dataset.reduce(0, lambda state, value: state + value).numpy())



8
3
22


In [7]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))

print(dataset1.element_spec)
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random.uniform([4]),
    tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))
print(dataset2.element_spec)
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
print(dataset3.element_spec)

TensorSpec(shape=(10,), dtype=tf.float32, name=None)
(TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))
(TensorSpec(shape=(10,), dtype=tf.float32, name=None), (TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None)))


In [19]:
def count(stop):
    i = 0
    while i<stop:
        yield i
        i += 1

ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types=tf.int32, output_shapes = (), )

for count_batch in ds_counter.repeat().batch(10).take(10):
    print(count_batch.numpy())


[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]


In [20]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)
images, labels = next(img_gen.flow_from_directory(flowers))
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

ds = tf.data.Dataset.from_generator(
    lambda: img_gen.flow_from_directory(flowers), 
    output_types=(tf.float32, tf.float32), 
    output_shapes=([32,256,256,3], [32,5])
)

ds.element_spec

for images, label in ds.take(1):
    print('images.shape: ', images.shape)
    print('labels.shape: ', labels.shape)


228827136/228813984 [==============================] - 88s 0us/step
Found 3670 images belonging to 5 classes.
float32 (32, 256, 256, 3)
float32 (32, 5)
Found 3670 images belonging to 5 classes.
images.shape:  (32, 256, 256, 3)
labels.shape:  (32, 5)


In [21]:
img_gen.flow_from_directory(flowers)

Found 3670 images belonging to 5 classes.


In [22]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                metrics=['accuracy'])

model.fit(fmnist_train_ds, epochs=2)


4431872/4422102 [==============================] - 2s 0us/step
Epoch 1/2
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5989 - accuracy: 0.7973: 0s - loss: 0.6005 - accuracy: 0.79
Epoch 2/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4629 - accuracy: 0.8420
